In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
#Open all source files
channel_volume = pd.read_csv('Untouched_Business_case/Data/Channel_Volume.csv')
location = pd.read_csv('Untouched_Business_case/Data/Locations.csv', delimiter=';').T
market_size = pd.read_csv('Untouched_Business_case/Data/Market_Sizes.csv')
company_share = pd.read_csv("company_share_cleaned.csv")
subcategories = pd.read_csv('Untouched_Business_case/Data/Subcategories.csv')
categories= pd.read_csv('Untouched_Business_case/Data/Categories.csv')

In [5]:
#Dataframe for Dragonyte only
company_share_copy = pd.read_csv("company_share_cleaned.csv")
company_share_copy_dragonyte = company_share_copy[company_share_copy["Global_Brand_Owner"]=="Dragonyte Brewery"]

In [6]:
#Transforming year into int so it works well in Tableau
company_share_copy_dragonyte["Year_text"] = company_share_copy_dragonyte["Year_text"].astype(int) 

/tmp/ipykernel_1003/681802043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_share_copy_dragonyte["Year_text"] = company_share_copy_dragonyte["Year_text"].astype(int)


In [7]:
#Market share of Dragonyte worldwide
volume_dragonyte = company_share_copy_dragonyte[company_share_copy_dragonyte["Location"]==8]["volume_litres"].sum()
volume_all = company_share_copy[company_share_copy["Location"]==8]["volume_litres"].sum()

In [8]:
#Market share of Dragonyte in the US
volume_dragonyte = company_share_copy_dragonyte[company_share_copy_dragonyte["Location"]==6]["volume_litres"].sum()
volume_all = company_share_copy[company_share_copy["Location"]==6]["volume_litres"].sum()
volume_dragonyte/volume_all*100

18.264474354784998

In [9]:
#How many types of beers does Dragonyte sell
len(company_share_copy_dragonyte["Subcategory_ID"].unique())

13

In [10]:
#How many locations Dragonyte is present in
len(company_share_copy_dragonyte["Location"].unique())

8

Computing the RSP/volume ratio absed on martket_size and RSP with market_share volumes

In [11]:
#computing RSP/volume ratio

market_size["RSP/volume"] = market_size["RSP"]/market_size["Volume"]

In [12]:
#Saving market sizes with new RSP/vol variable
market_size.to_csv("market_size_rsp_volume.csv")

In [13]:
#Here either we use median, sum or mean as aggregator it makes no difference
market_size_grouped = market_size[market_size["Location"]!=8.0].groupby(["Year","Location", "Subcategory"],as_index=False)["RSP/volume"].median()
market_size_grouped

,Year,Location,Subcategory,RSP/volume
0,2016,1,1,11.148148
1,2016,1,2,12.658537
2,2016,1,3,2.561224
3,2016,1,4,2.153991
4,2016,1,6,0.009309
...,...,...,...,...
1216,2026,7,11,4.812133
1217,2026,7,12,1.239910
1218,2026,7,13,5.166435
1219,2026,7,16,0.009438


In [14]:
#total volume worlwide and displaying it as a df
company_share_grouped = company_share[company_share["Location"]!=8.0].groupby(["Year_text","Location", "Subcategory_ID"],as_index=False)["volume_litres"].sum()
company_share_grouped

,Year_text,Location,Subcategory_ID,volume_litres
0,2016.0,1.0,1.0,2.667644e+07
1,2016.0,1.0,2.0,4.085848e+07
2,2016.0,1.0,3.0,9.840531e+07
3,2016.0,1.0,4.0,6.782176e+10
4,2016.0,1.0,6.0,1.364653e+08
...,...,...,...,...
673,2021.0,7.0,13.0,2.804911e+10
674,2021.0,7.0,14.0,1.166836e+10
675,2021.0,7.0,15.0,1.882811e+06
676,2021.0,7.0,16.0,4.936165e+05


In [15]:
#filtering market share to have volumes for Dragonyte Brewery
volumes_dragonyte = company_share[(company_share["Location"] != 8.0) & (company_share["Global_Brand_Owner"] =="Dragonyte Brewery")].groupby(["Year_text","Location", "Subcategory_ID"],as_index=False)["volume_litres"].sum()
volumes_dragonyte

,Year_text,Location,Subcategory_ID,volume_litres
0,2016.0,1.0,4.0,9.134265e+07
1,2016.0,1.0,8.0,9.134265e+07
2,2016.0,1.0,10.0,2.037280e+07
3,2016.0,1.0,12.0,7.088152e+07
4,2016.0,1.0,13.0,9.134265e+07
...,...,...,...,...
297,2021.0,7.0,9.0,1.404519e+08
298,2021.0,7.0,10.0,2.053907e+08
299,2021.0,7.0,11.0,6.333683e+08
300,2021.0,7.0,13.0,9.974008e+08


In [16]:
#Transforming subcategory into an INT so that it is usable in Tableau
company_share_grouped["Subcategory_ID"] = company_share_grouped["Subcategory_ID"].astype(int)

In [17]:
#Multiplying the RSP/volume ratio from market_size by volumes from market_share
for index, value in volumes_dragonyte["volume_litres"].items():
    for ind, val in market_size_grouped["RSP/volume"].items():
        if market_size_grouped.loc[ind, "Year"] == volumes_dragonyte.loc[index, "Year_text"] and market_size_grouped.loc[ind, "Subcategory"] == volumes_dragonyte.loc[index, "Subcategory_ID"] and market_size_grouped.loc[ind, "Location"] == volumes_dragonyte.loc[index, "Location"]:
            volumes_dragonyte.loc[index, "RSP"] = volumes_dragonyte.loc[index, "volume_litres"]*market_size_grouped.loc[ind, "RSP/volume"]

In [18]:
#Transforming subcategory into an INT so that it is usable in Tableau
volumes_dragonyte["Location"] = volumes_dragonyte["Location"].astype(int)
volumes_dragonyte["Subcategory_ID"] = volumes_dragonyte["Subcategory_ID"].astype(int)

In [19]:
#Saving the file as csv
volumes_dragonyte.to_csv("Dragonyte_RSP_vols.csv")

In [20]:
#Multiplying the RSP/volume ratio from market_size by volumes from market_share FOR DRAGONYTE ONLY
for index, value in company_share_grouped["volume_litres"].items():
    for ind, val in market_size_grouped["RSP/volume"].items():
        if market_size_grouped.loc[ind, "Year"] == company_share_grouped.loc[index, "Year_text"] and market_size_grouped.loc[ind, "Subcategory"] == company_share_grouped.loc[index, "Subcategory_ID"] and market_size_grouped.loc[ind, "Location"] == company_share_grouped.loc[index, "Location"]:
            company_share_grouped.loc[index, "RSP"] = company_share_grouped.loc[index, "volume_litres"]*market_size_grouped.loc[ind, "RSP/volume"]

In [21]:
#grouping by location
company_share_grouped_new = company_share_grouped.groupby("Location",as_index=False)["RSP"].sum()
company_share_grouped_new

,Location,RSP
0,1.0,4.401856e+12
1,2.0,2.712256e+11
2,3.0,1.168579e+12
3,4.0,2.872128e+12
4,5.0,1.068519e+12
5,6.0,2.909490e+12
6,7.0,3.008704e+12


In [22]:
#Making sure subcategory ID is an int so it will be usable in Tableau
company_share_grouped["Subcategory_ID"]=company_share_grouped["Subcategory_ID"].astype(int)

In [23]:
#Making sure location is an int so it will be usable in Tableau
company_share_grouped["Location"]=company_share_grouped["Location"].astype(int)

In [24]:
#Saving as csv
company_share_grouped.to_csv("company_share_RSP_final.csv")

In [25]:
#Saving as csv
company_share_grouped_new.to_csv("company_share_RSP_V2.csv")

Transforming Volume and RSP into intelligible data (dots used for thousands separators and decimals)

In [26]:
market_size_copy = market_size.copy()

In [27]:
#we ended up not using this but instead taking the data from market_share because we do not have the info of either it is in thousands or in millions which messes the data

market_size_copy["Volume"] = market_size_copy["Volume"].astype(str)
for index,value in market_size_copy["Volume"].items():
    a = value.split('.')
    if a[1] == '0':
        market_size_copy.loc[index,"Volume"] = float(a[0])
    else :
        market_size_copy.loc[index,"Volume"] = float(value)*1000

In [28]:
#Same conclusion as above
market_size_copy["RSP"] = market_size_copy["RSP"].astype(str)

for index,value in market_size_copy["RSP"].items():
    a = value.split('.')
    if a[1] == '0':
        market_size_copy.loc[index,"RSP"] = float(a[0])
    else :
        market_size_copy.loc[index,"RSP"] = float(value)*1000


Cleaning the dataset MARKET_SIZE - we ended up using only the RSP/vol ratio

In [29]:
market_size["Volume"] = market_size["Volume"].astype(float)

In [30]:
market_size["RSP"] = market_size["RSP"]*1000

In [31]:
market_size["Volume"] = market_size["Volume"].astype(int)
market_size["RSP"] = market_size["RSP"].astype(int)

In [32]:
market_size = market_size.dropna(axis = 0, how = 'all')

In [33]:
market_size.rename(columns = {'RSP':'RSP (USD million)'}, inplace = True)
market_size.rename(columns = {'Volume':'Volume (Million liters)'}, inplace = True)

In [34]:
#Transforming the year string into datetime 
market_size['Year_date'] = pd.to_datetime(market_size['Year_date'])

/tmp/ipykernel_1003/124482147.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  market_size['Year_date'] = pd.to_datetime(market_size['Year_date'])


In [35]:
#Verifying that every date is good
a = market_size['Year_date'].dt.year-market_size['Year']
a.isnull().count()

1221

In [36]:
#Verifying that all unit are the same, -> they are
market_size['Unit'].unique()

array(['USD million'], dtype=object)

In [37]:
market_size.to_csv("market_size_cleaned_bis.csv")

In [38]:
market_size_bis = pd.read_csv("market_size_cleaned_bis.csv")

SUBCATEGORIES

In [40]:
subcategories = subcategories.dropna(axis = 0, how = 'all')
subcategories.to_csv("subcategories_cleaned.csv")

In [41]:
categories = categories.dropna(axis = 0, how = 'all')
categories.to_csv("categories_cleaned.csv")

LOCATION

In [46]:
location = location.dropna(axis = 0, how = 'all')
location.columns = location.iloc[0]
location = location.iloc[1:]
location.to_csv("location.csv")

id,Region,Country
1,Asia Pacific,Asia Pacific
2,Australasia,Australasia
3,Eastern Europe,Eastern Europe
4,Latin America,Latin America
5,Middle East and Africa,Middle East and Africa
6,North America,North America
7,Western Europe,Western Europe
8,World,World
